In [4]:
!nvidia-smi

Thu Oct 26 18:03:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip -q install transformers peft accelerate bitsandbytes getpass4 torchrl sentencepiece

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pickle
with open("/content/drive/MyDrive/robotics+llm/Data/image_embeddings.pickle", "rb") as f:
  data = pickle.load(f)

In [8]:
import torch

img_embeddings = []
next_img_embeddings = []
actions = []
instructions = []
for key in data:
  item = data[key]
  img_embeddings.append(item[0])
  next_img_embeddings.append(item[2])
  actions.append(torch.tensor(item[1]).reshape(-1, 2))
  instructions.append(item[3])

img_embeddings_t= torch.cat(img_embeddings, dim=0)
next_img_embeddings_t = torch.cat(next_img_embeddings, dim=0)
actions_t = torch.cat(actions, dim=0)

In [9]:
img_embeddings_t.shape, next_img_embeddings_t.shape, actions_t.shape, len(instructions)

(torch.Size([252, 768]), torch.Size([252, 768]), torch.Size([252, 2]), 252)

In [10]:
from getpass import getpass
hf_token = getpass("Huggingface access token")

Huggingface access token··········


In [11]:
import bitsandbytes as bnb
import os
import torch

from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig

device_map = "auto"

base_model = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = LlamaForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map=device_map,
    token=hf_token
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [12]:
tokenizer = LlamaTokenizer.from_pretrained(base_model, token=hf_token)
tokenizer.add_special_tokens(
    {
        "pad_token": "<PAD>",
    }
)
model.config.pad_token_id = tokenizer.pad_token_id
special_tokens_dict = {'additional_special_tokens': ['<IMG>', '<X>', '<Y>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(32004, 4096)

In [13]:
tokenizer.get_added_vocab()

{'<unk>': 0,
 '<s>': 1,
 '</s>': 2,
 '<PAD>': 32000,
 '<Y>': 32001,
 '<X>': 32002,
 '<IMG>': 32003}

In [14]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32004, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [15]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
    get_peft_config,
    PromptTuningInit,
    PromptTuningConfig,
    TaskType,
    PeftType
)

lora_r = 16
lora_alpha = 32
lora_dropout = 0.1
lora_target_modules = ".*(q_proj|k_proj|v_proj|o_proj|gate_proj|down_proj|up_proj|lm_head)$"
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 40,554,560 || all params: 6,779,002,944 || trainable%: 0.5982378284094754


In [16]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32004, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
              

In [17]:
# Labels
import torchrl

class ActionTokenizer:
  def __init__(self,
               action_spec: dict,
               vocab_size: int,
               action_order=None):
    self.action_spec = action_spec
    self.vocab_size = vocab_size
    if action_order is None:
      self.action_order = self.action_spec.keys()
    else:
      for action in action_order:
        if action not in self.action_spec.keys():
          raise ValueError(f"actions: {action} not found in action_spec: {action_spec.keys()}")
        assert action in self.action_spec.keys()
      self.action_order = action_order

  def tokenize(self, actions):
    """Tokenizes an action."""
    action_tokens = []
    for k in self.action_order:
      a = actions[k]  # a is [batch, actions_size]
      spec = self.action_spec[k]
      a = torch.clip(a, spec.space.low, spec.space.high)
      # Normalize the action [batch, actions_size]
      token = (a - spec.space.low) / (spec.space.high - spec.space.low)
      # Bucket and discretize the action to vocab_size, [batch, actions_size]
      token = (token * (self.vocab_size -1))
      token = token.type(torch.int32)
      action_tokens.append(token)
    # Append all actions, [batch, all_actions_size]
    action_tokens = torch.cat(action_tokens, axis=-1)
    return action_tokens

  def detokenize(self, action_tokens):
    """Detokenizes an action."""
    action = {}
    token_index = 0
    for k in self.action_order:
      spec = self.action_spec[k]
      action_dim = spec.shape[0]
      actions = []
      for _ in range(action_dim):
        spec_index = token_index % action_dim
        a = action_tokens[..., token_index:token_index + 1]
        a = a.type(torch.float32)
        a = a / (self.vocab_size - 1)
        a = (a * (spec.space.high[spec_index] - spec.space.low[spec_index])) + spec.space.low[spec_index]
        actions.append(a)
        token_index += 1
      action[k] = torch.cat(actions, axis=-1)
    return action


/usr/local/lib/python3.10/dist-packages/torchrl/__init__.py:32: UserWarning: failed to set start method to spawn, and current start method for mp is fork.
  warn(


In [18]:
# action_spec = {
#     "dX": torchrl.data.BoundedTensorSpec(low=-10., high=10., shape=(1,), dtype=torch.float32),
#     "dY": torchrl.data.BoundedTensorSpec(low=-10., high=10., shape=(1,), dtype=torch.float32)
# }
# vocab_size = 1000000
# action_tokenizer = ActionTokenizer(action_spec, vocab_size)

In [19]:
# action = { "dX": torch.Tensor([-0.1]), "dY": torch.Tensor([1.2]) }
# toks = action_tokenizer.tokenize(action)
# toks

In [20]:
# action_tokenizer.detokenize(toks)

In [146]:
from torch.utils.data import Dataset

class LlameDataset(Dataset):
  def __init__(self,
               img_embeddings,
               next_image_embeddings,
               actions,
               instructions,
               tokenizer):
    self.img_embeddings = img_embeddings
    self.next_image_embeddings = next_image_embeddings
    self.actions = actions
    self.instructions = instructions
    self.tokenizer = tokenizer

  def __len__(self):
    return self.img_embeddings.shape[0]

  def __getitem__(self, idx):
    img_emb = self.img_embeddings[idx]
    next_img_emb = self.next_image_embeddings[idx]
    act = self.actions[idx]
    inst = self.instructions[idx]
    ins_prefix = "<s>Given "
    ins_suffix = f", {inst} <X> <Y></s>"
    ins_prefix_tokens = self.tokenizer(ins_prefix, return_tensors="pt", add_special_tokens=False)
    ins_suffix_tokens = self.tokenizer(ins_suffix, return_tensors="pt", padding=False, max_length=100, add_special_tokens=False)
    ins_placeholder = f"{ins_prefix}<IMG>{ins_suffix}"
    ins_tokens = self.tokenizer(ins_placeholder, return_tensors="pt", add_special_tokens=False, padding=False, max_length=105)

    return {
        "current_img_emb": img_emb,
        "next_img_emb": next_img_emb,
        "ins_prefix_tokens": ins_prefix_tokens,
        "ins_suffix_tokens": ins_suffix_tokens,
        "ins_placeholder_tokens": ins_tokens,
        "action_x": act[0],
        "action_y": act[1]
    }

In [147]:
ds = LlameDataset(
    img_embeddings_t[:10],
    next_img_embeddings_t[:10],
    actions_t[:10],
    instructions[:10],
    tokenizer
)

In [148]:
len(ds)

10

In [149]:
import torch.nn as nn
vit_to_llm = nn.Linear(768, 4096).to("cuda")

X_head = nn.Sequential(
    nn.Linear(4096, 256),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(256, 1)
).to("cuda")
Y_head = nn.Sequential(
    nn.Linear(4096, 256),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(256, 1)
).to("cuda")
next_img_head = nn.Sequential(
    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(1024, 1024),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(1024, 768)
).to("cuda")

In [150]:
# dx = act_tokens[0].item()
# dy = act_tokens[1].item()
# ins_prefix = "<s>Given "
# ins_suffix = f", move the red block to the green block. Actions: <X>|<Y></s>"
# ins_prefix_tokens = tokenizer(ins_prefix, return_tensors="pt", add_special_tokens=False)
# ins_suffix_tokens = tokenizer(ins_suffix, return_tensors="pt", add_special_tokens=False, padding="max_length", max_length=100)

In [151]:
# ins_suffix

In [152]:
# ins_placeholder = f"{ins_prefix}<IMG>{ins_suffix}"
# ins_tokens = tokenizer(ins_placeholder, return_tensors="pt", add_special_tokens=False, padding="max_length", max_length=105)
# ins_tokens["input_ids"].shape

In [153]:
# ins_prefix_tokens

In [154]:
# ins_suffix_tokens["input_ids"].shape

In [155]:
# ins_prefix_embeddings = model.model.embed_tokens(ins_prefix_tokens["input_ids"])
# ins_suffix_embeddings = model.model.embed_tokens(ins_suffix_tokens["input_ids"])

In [156]:
# img_embeddings = vit_to_llm(_x[0].unsqueeze(0).to("cuda")).to("cuda")
# img_embeddings.shape

In [157]:
# ins_prefix_embeddings.shape

In [158]:
# ins_suffix_embeddings.shape

In [159]:
# embeddings = torch.cat([ins_prefix_embeddings, img_embeddings, ins_suffix_embeddings], dim=1)
# embeddings.shape

In [160]:
# labels = ins_tokens["input_ids"]
# labels

In [161]:
# labels.shape

In [162]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    ds,
    batch_size=1,
    shuffle=True,
    num_workers=1
)


In [163]:
import gc

In [164]:
from tqdm.auto import tqdm
from torch.optim import AdamW
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import torch.nn.functional as F
import random
from torch.autograd import Variable
from itertools import chain

lr = 1e-3

mse_loss = torch.nn.MSELoss()

trainable_models = chain(
    model.parameters(),
    vit_to_llm.parameters(),
    X_head.parameters(),
    Y_head.parameters(),
    next_img_head.parameters()
)

optimizer = AdamW(params=trainable_models, lr=lr)

losses_total = []
losses_vit = []
losses_x = []
losses_y = []
losses_next = []
losses_ce = []

grads_total = []
grads_vit = []
grads_x = []
grads_y = []
grads_next = []

num_epochs = 56
for epoch in tqdm(range(num_epochs)):
  gc.collect()
  for i, batch in tqdm(enumerate(train_dataloader)):
      mapped_img_embeddings = vit_to_llm(batch["current_img_emb"].reshape(batch["current_img_emb"].shape[0], 1, -1).to("cuda"))
      ins_prefix_embeddings = model.base_model.model.model.embed_tokens(batch["ins_prefix_tokens"]["input_ids"].squeeze(1))
      ins_suffix_embeddings = model.base_model.model.model.embed_tokens(batch["ins_suffix_tokens"]["input_ids"].squeeze(1))
      embeddings = torch.cat([ins_prefix_embeddings, mapped_img_embeddings, ins_suffix_embeddings], dim=1)

      out = model(inputs_embeds=embeddings.half(), labels=batch["ins_placeholder_tokens"]["input_ids"], output_hidden_states=True)

      last_hidden_state = out.hidden_states[-1][:, -1, :].type(torch.float32)
      x_out = X_head(last_hidden_state)
      y_out = Y_head(last_hidden_state)
      next_img_out = next_img_head(last_hidden_state)

      ce_loss = out.loss
      x_loss = mse_loss(batch["action_x"].to("cuda"), x_out)
      y_loss = mse_loss(batch["action_y"].to("cuda"), y_out)
      next_img_loss = mse_loss(batch["next_img_emb"].to("cuda"), next_img_out)

      loss =  ce_loss + x_loss + y_loss + next_img_loss

      # loss.backward(retain_graph=True)

      # if step % 100 == 0:
      #     G_loss = torch.autograd.grad(loss, W.parameters(), retain_graph=True, create_graph=True)
      #     G_loss_norm = torch.norm(G_loss[0], 2) # G_loss[0] = weights G_loss[1] = bias
      #     G_triplet = torch.autograd.grad(loss_triplet, W.parameters(), retain_graph=True, create_graph=True)
      #     G_triplet_norm = torch.norm(G_triplet[0], 2)
      #     G_ce = torch.autograd.grad(loss_ce, W.parameters(), retain_graph=True, create_graph=True)
      #     G_ce_norm = torch.norm(G_ce[0], 2)

      #     grads_total.append(G_loss_norm.item())
      #     grads_triplets.append(G_triplet_norm.item())
      #     grads_ce.append(G_ce_norm.item())

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      # if step % 100 == 0:
      losses_total.append(loss.item())
      losses_ce.append(ce_loss.item())
      losses_x.append(x_loss.item())
      losses_y.append(y_loss.item())
      losses_next.append(next_img_loss.item())

  avg_total_loss = sum(losses_total) / ((epoch + 1) * len(train_dataloader))
  avg_ce_loss = sum(losses_ce) / ((epoch + 1) * len(train_dataloader))
  avg_x_loss = sum(losses_x) / ((epoch + 1) * len(train_dataloader))
  avg_y_loss = sum(losses_y) / ((epoch + 1) * len(train_dataloader))
  avg_next_loss = sum(losses_next) / ((epoch + 1) * len(train_dataloader))

  print(f"Total: {avg_total_loss}  CE: {avg_ce_loss} X: {avg_x_loss}  Y: {avg_y_loss} Next: {avg_next_loss}")

  0%|          | 0/56 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Total: 21.834269952774047  CE: 4.830816161632538 X: 6.717249272763729  Y: 10.025165058672428 Next: 0.26103938519954684


...

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Total: 2.684482734745199  CE: 1.6304808353294025 X: 0.39367076225907033  Y: 0.6392996486325346 Next: 0.021031479576433247


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0it [00:00, ?it/s]

Total: 2.644657864049077  CE: 1.6092118011521441 X: 0.3866464485047046  Y: 0.6279946559650149 Next: 0.020804949650274854


In [167]:
# for epoch in tqdm(range(50, 56)):
#   gc.collect()
#   for i, batch in tqdm(enumerate(train_dataloader)):
#       mapped_img_embeddings = vit_to_llm(batch["current_img_emb"].reshape(batch["current_img_emb"].shape[0], 1, -1).to("cuda"))
#       ins_prefix_embeddings = model.base_model.model.model.embed_tokens(batch["ins_prefix_tokens"]["input_ids"].squeeze(1))
#       ins_suffix_embeddings = model.base_model.model.model.embed_tokens(batch["ins_suffix_tokens"]["input_ids"].squeeze(1))
#       embeddings = torch.cat([ins_prefix_embeddings, mapped_img_embeddings, ins_suffix_embeddings], dim=1)

#       out = model(inputs_embeds=embeddings.half(), labels=batch["ins_placeholder_tokens"]["input_ids"], output_hidden_states=True)

#       last_hidden_state = out.hidden_states[-1][:, -1, :].type(torch.float32)
#       x_out = X_head(last_hidden_state)
#       y_out = Y_head(last_hidden_state)
#       next_img_out = next_img_head(last_hidden_state)

#       ce_loss = out.loss
#       x_loss = mse_loss(batch["action_x"].to("cuda"), x_out)
#       y_loss = mse_loss(batch["action_y"].to("cuda"), y_out)
#       next_img_loss = mse_loss(batch["next_img_emb"].to("cuda"), next_img_out)

#       loss = ce_loss + x_loss + y_loss + next_img_loss

#       # loss.backward(retain_graph=True)

#       # if step % 100 == 0:
#       #     G_loss = torch.autograd.grad(loss, W.parameters(), retain_graph=True, create_graph=True)
#       #     G_loss_norm = torch.norm(G_loss[0], 2) # G_loss[0] = weights G_loss[1] = bias
#       #     G_triplet = torch.autograd.grad(loss_triplet, W.parameters(), retain_graph=True, create_graph=True)
#       #     G_triplet_norm = torch.norm(G_triplet[0], 2)
#       #     G_ce = torch.autograd.grad(loss_ce, W.parameters(), retain_graph=True, create_graph=True)
#       #     G_ce_norm = torch.norm(G_ce[0], 2)

#       #     grads_total.append(G_loss_norm.item())
#       #     grads_triplets.append(G_triplet_norm.item())
#       #     grads_ce.append(G_ce_norm.item())

#       loss.backward()
#       optimizer.step()
#       optimizer.zero_grad()

#       # if step % 100 == 0:
#       losses_total.append(loss.item())
#       losses_ce.append(ce_loss.item())
#       losses_x.append(x_loss.item())
#       losses_y.append(y_loss.item())
#       losses_next.append(next_img_loss.item())

#   avg_total_loss = sum(losses_total) / ((epoch + 1) * len(train_dataloader))
#   avg_ce_loss = sum(losses_ce) / ((epoch + 1) * len(train_dataloader))
#   avg_x_loss = sum(losses_x) / ((epoch + 1) * len(train_dataloader))
#   avg_y_loss = sum(losses_y) / ((epoch + 1) * len(train_dataloader))
#   avg_next_loss = sum(losses_next) / ((epoch + 1) * len(train_dataloader))

#   print(f"Total: {avg_total_loss}  CE: {avg_ce_loss} X: {avg_x_loss}  Y: {avg_y_loss} Next: {avg_next_loss}")

In [168]:
test_ds = LlameDataset(
    img_embeddings_t[11:20],
    next_img_embeddings_t[11:20],
    actions_t[11:20],
    instructions[11:20],
    tokenizer
)

In [169]:
test_dl = DataLoader(
    test_ds,
    batch_size=1,
    num_workers=1
)

In [170]:
_, _d = next(enumerate(test_dl))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [171]:
_img_emb = vit_to_llm(_d["current_img_emb"].reshape(_d["current_img_emb"].shape[0], 1, -1).to("cuda"))
p_emb = model.base_model.model.model.embed_tokens(_d["ins_prefix_tokens"]["input_ids"].squeeze(1))
s_emb = model.base_model.model.model.embed_tokens(_d["ins_suffix_tokens"]["input_ids"].squeeze(1))
_merged_emb = torch.cat([p_emb, _img_emb, s_emb], dim=1)
print(_merged_emb.shape)

torch.Size([1, 34, 4096])


In [172]:
gc.collect()

214

In [173]:
_gen = model.generate(
    inputs_embeds=_merged_emb,
    return_dict_in_generate=True,
    output_scores=True,
    output_hidden_states=True,
    max_new_tokens=50
)

In [174]:
_gen

SampleDecoderOnlyOutput(sequences=tensor([[    1,   278,   278,   278,   278, 29871,   278,   278,   278,   278,
         29871,   278, 29871,   278, 11137,   278, 32001, 11137, 29871, 32002,
         29871, 29871, 29871, 32001, 29871, 29871, 29871, 32001,   278, 32003,
           411,   278,   278, 29871,   278, 11137, 32001, 12841, 32001, 29871,
           278, 29871, 29871,  2654, 32001,   411, 12841, 29871, 29871, 29871,
           278]], device='cuda:0'), scores=(tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[   -inf,    -inf,    -inf,  ...,    -inf, 24.0454,    -inf]],
       device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tenso

In [175]:
hs = _gen.hidden_states
seq = _gen.sequences

In [176]:
seq

tensor([[    1,   278,   278,   278,   278, 29871,   278,   278,   278,   278,
         29871,   278, 29871,   278, 11137,   278, 32001, 11137, 29871, 32002,
         29871, 29871, 29871, 32001, 29871, 29871, 29871, 32001,   278, 32003,
           411,   278,   278, 29871,   278, 11137, 32001, 12841, 32001, 29871,
           278, 29871, 29871,  2654, 32001,   411, 12841, 29871, 29871, 29871,
           278]], device='cuda:0')

In [177]:
tokenizer.batch_decode(seq, skip_special_tokens=True)

['the the the the  the the the the  the  the pent the pent        the with the the  the pentagon  the   red withagon    the']

In [178]:
_d["action_x"]

tensor([0.0024])

In [179]:
_d["action_y"]

tensor([-0.0175])

In [180]:
_next_img_e = _d["next_img_emb"].to("cuda")

In [181]:
for i in range(10):
  last_hidden_state = hs[i][-1][:, -1, :]
  _out_x = X_head(last_hidden_state)
  _out_y = Y_head(last_hidden_state)
  _out_next = next_img_head(last_hidden_state)
  print(_out_x)
  print(_out_y)
  print(nn.functional.cosine_similarity(_out_next, _next_img_e))
  print("-" * 10)


tensor([[0.0728]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.1484]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0.9137], device='cuda:0', grad_fn=<SumBackward1>)
----------
tensor([[0.0041]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0087]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0.9155], device='cuda:0', grad_fn=<SumBackward1>)
----------
tensor([[-2.3133e-05]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0096]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0.8902], device='cuda:0', grad_fn=<SumBackward1>)
----------
tensor([[0.0128]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0422]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0.9179], device='cuda:0', grad_fn=<SumBackward1>)
----------
tensor([[0.0597]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0874]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0.9207], device='cuda:0', grad_fn=<SumBackward1>)
----------
tensor([[0.5069]], device='c